# 🚀  Project: Smart Inventory & Fruit Counting System (YOLOv26)

**Scenario**:

Detecting and counting apples moving on a conveyor belt using a vertical virtual line. Method: Custom Coordinate-Based Counting (Manual Buffer Logic).

NOTE: I recommend running this project notebook with Google Colab using a T4 GPU.

## 🍎 Smart Inventory & Object Tracking System (YOLOv26)

📋 Project Overview

This project demonstrates a real-time Computer Vision solution designed for automated inventory management in industrial environments. Using the state-of-the-art YOLOv26 object detection architecture, the system identifies, tracks, and counts items (e.g., fruits) moving on a conveyor belt with high precision.Unlike standard counting methods that rely on simple line-crossing algorithms, this project implements a Custom Buffer Zone Logic to ensure 100% accuracy. This approach mitigates common issues such as double-counting caused by video jitter or object occlusion.

🎯 Key Features

Object Detection: Fine-tuned YOLOv26 model to detect specific objects (e.g., Apples) from a custom dataset.Persistent Tracking: Utilizes advanced tracking algorithms (BoT-SORT/ByteTrack) to assign unique IDs to moving objects.

Robust Counting Logic: Implements a "Safety Corridor" (Buffer Zone) mechanism. Objects are counted only when their centroid coordinates stabilize within a specific vertical range ($X \pm Offset$).

Real-Time Dashboard: Displays live analytics including bounding boxes, tracking IDs, and total count directly on the video feed.

🛠️ Tech Stack & ToolsDeep

Learning: Ultralytics YOLOv26 (Transfer Learning)

Computer Vision: OpenCV (cv2) for frame processing and visualization

Data Management: Roboflow API (Automated Dataset Ingestion)

Infrastructure: Google Colab (GPU Acceleration)

Project Link: https://github.com/fhattat/YOLOv26-Smart-Inventory-Counter

## STEP 1: Environment Setup & Library Installation

We need to install the ultralytics library for YOLOv26 models and roboflow to fetch the dataset.

In [1]:
# STEP 1: Install Dependencies
# We use the '!' command to run terminal commands in Colab.

!pip install ultralytics roboflow
!pip install opencv-python-headless  # OpenCV for video processing

import cv2
import torch
from ultralytics import YOLO
import os
import numpy as np
from IPython.display import display, Image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 126.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.13.0.90
    Uninstalling opencv-python-headless-4.13.0.90:
      Successfully uninstalled opencv-python-headless-4.13.0.90
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings wit

In [2]:
# Check GPU availability for faster training
print(f"✅ Setup Completed.")
print(f"🔥 GPU Available: {torch.cuda.is_available()}")
!nvidia-smi

✅ Setup Completed.
🔥 GPU Available: True
Thu Jan 29 20:47:34 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+------

## STEP 2: Download Dataset (Roboflow)

We are pulling the "Fruit Detection" dataset directly from Roboflow Universe using your API key.

Link for the data set used:

https://universe.roboflow.com/toronto-metropolitan-university/fruit-detection-vibck

In [4]:
# STEP 2: Load Dataset from Roboflow
from roboflow import Roboflow

try:
    rf = Roboflow(api_key="o5YVlzSvoMb7P7brUGmp")   # API_KEY_GOES_HERE FROM ROBOFLOW
    project = rf.workspace("toronto-metropolitan-university").project("fruit-detection-vibck")
    version = project.version(4)
    dataset = version.download("yolo26")

    dataset_yaml = dataset.location + "/data.yaml"
    print(f"✅ Data set path: {dataset_yaml}")
except:
    print("⚠️ API Key not entered. Training step will be skipped or demo will be performed.")
    dataset_yaml = None  # Defining the dataset path variable for later use

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Fruit-Detection-4 in yolo26:: 100%|██████████| 13376/13376 [00:01<00:00, 6797.74it/s]


✅ Data set path: /content/Fruit-Detection-4/data.yaml


## STEP 3: Model Training (Transfer Learning)

We will fine-tune the yolo26m (Medium) model on our fruit dataset.
For high-level training, “yolo26x.pt” can be selected.

In [5]:
# STEP 3: Train YOLO Model
# We load the pre-trained YOLO26m model and train it on our new data.

# Load a pre-trained model
model = YOLO('yolo26m.pt')

if dataset_yaml:
    print("🚀 Training Started... (This may take a while)")

    results = model.train(
        data=dataset_yaml,  # Path to dataset config
        epochs=20,          # Number of training cycles
        imgsz=640,          # Image resolution
        batch=16,           # Batch size
        plots=True,         # Generate training graphs
        name='fruit_counter_final' # Name of the project folder
    )
    print("✅ Training Completed Successfully.")

    # Path to the best performing weights
    best_model_path = '/content/runs/detect/fruit_counter_final/weights/best.pt'

else:
    print("❌ Dataset not found. Cannot start training.")
    # Fallback to pre-trained model if training fails
    best_model_path = 'yolo26m.pt'

🚀 Training Started... (This may take a while)
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Fruit-Detection-4/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26m.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=fruit_counter_final, nbs=64, nms=False, opset=None, optimize=Fa

## STEP 4: Inference Configuration (Video Setup)

Define the input video path and output settings. Note: Make sure to upload your test_video.mp4 to the Colab files section before running this. This test video was uploaded to GitHub Profile

https://github.com/fhattat/YOLOv26-Smart-Inventory-Counter

**NOTE**: This test video was created by Google Veo3

In [6]:
# STEP 4: Define Video Paths & Load Trained Model

# Input video file (Upload this to Colab Files on the left)
input_video_path = "/content/test_video.mp4"   # video must be uploaded to left pane
output_video_path = "/content/final_smart_counter.mp4"

# Check if video exists
if not os.path.exists(input_video_path):
    print(f"❌ Error: File '{input_video_path}' not found.")
    print("Please upload 'test_video.mp4' to the file section.")
else:
    print(f"✅ Video Found: {input_video_path}")

# Load the custom trained model
print(f"🧠 Loading Model from: {best_model_path}")
model = YOLO(best_model_path)

✅ Video Found: /content/test_video.mp4
🧠 Loading Model from: /content/runs/detect/fruit_counter_final/weights/best.pt


## STEP 5: The Core Logic (Manual Counting)

This is the most critical part. We implement a Buffer Zone mechanism.

**Vertical Line**: Placed in the center of the screen.

**Offset**: A safe area (buffer) around the line.

**Logic**: If (Line - Offset) < Object_Center < (Line + Offset), we count it.

In [8]:
# STEP 5: Process Video & Count Objects (Manual Logic)

# Open Video Capture
cap = cv2.VideoCapture(input_video_path)

# Get Video Properties
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize Video Writer to save output
video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# --- COUNTING CONFIGURATION ---
center_x = w // 2        # X-Coordinate of the vertical line
offset = 30              # Buffer zone in pixels (Sensitivity)
total_count = 0          # Initialize counter
counted_ids = []         # List to keep track of counted Object IDs

print(f"⚙️ Video Resolution: {w}x{h}")
print(f"📏 Counting Line at X={center_x} (Buffer: +/- {offset}px)")
print("🎬 Processing Video... Please wait.")

frame_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break # End of video

    frame_count += 1

    # 1. Object Tracking
    # conf=0.15: Filter out weak detections
    # persist=True: Essential for tracking objects across frames
    results = model.track(frame, persist=True, verbose=False, conf=0.15, iou=0.5)

    # Get detection boxes and IDs
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xywh.cpu() # Get box coordinates (x, y, w, h)
        track_ids = results[0].boxes.id.int().cpu().tolist() # Get unique tracking IDs

        # Loop through each detected object
        for box, track_id in zip(boxes, track_ids):
            x, y, w_box, h_box = box

            # Calculate Centroid (Center point of the object)
            cx = int(x)
            cy = int(y)

            # --- COUNTING LOGIC (The "Magic" Part) ---
            # Check if the object is inside the vertical buffer zone
            if (center_x - offset) < cx < (center_x + offset):

                # Check if this specific ID has been counted before
                if track_id not in counted_ids:
                    total_count += 1
                    counted_ids.append(track_id) # Add to history

                    # Visual Feedback: Flash a Green Line when counted
                    cv2.line(frame, (center_x, 0), (center_x, h), (0, 255, 0), 4)
                    print(f"🍎 Counted! ID: {track_id} | Total: {total_count}")

            # Draw a small red dot at the center of the object
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

    # 2. Visualization (Drawing on Frame)

    # Draw Bounding Boxes (from Model)
    frame = results[0].plot()

    # Draw The Reference Lines (Blue)
    # Main Center Line
    cv2.line(frame, (center_x, 0), (center_x, h), (255, 0, 0), 2)
    # Buffer Zone Limits (Thinner Cyan Lines)
    cv2.line(frame, (center_x - offset, 0), (center_x - offset, h), (255, 255, 0), 1)
    cv2.line(frame, (center_x + offset, 0), (center_x + offset, h), (255, 255, 0), 1)

    # 3. Dashboard Display (Bottom Left)
    text = f"Total Count: {total_count}"

    # Create a background rectangle for better readability
    (text_w, text_h), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)

    # Rectangle Coordinates (Bottom Left)
    rect_start = (20, h - 90)
    rect_end = (40 + text_w, h - 30)

    # Draw Black Background Rectangle
    cv2.rectangle(frame, rect_start, rect_end, (0, 0, 0), -1)

    # Draw Text (Yellow)
    cv2.putText(frame, text, (30, h - 45), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 3)

    # Save Frame to Output Video
    video_writer.write(frame)

    # Progress Log
    if frame_count % 30 == 0:
        print(f"⏳ Processed Frame {frame_count} | Current Count: {total_count}")

# Release Resources
cap.release()
video_writer.release()
# cv2.destroyAllWindows() <--- REMOVED TO PREVENT COLAB ERROR

print("-" * 40)
print(f"✅ SUCCESS! Video saved to: {output_video_path}")

⚙️ Video Resolution: 1920x1080
📏 Counting Line at X=960 (Buffer: +/- 30px)
🎬 Processing Video... Please wait.
🍎 Counted! ID: 789 | Total: 1
🍎 Counted! ID: 833 | Total: 2
⏳ Processed Frame 30 | Current Count: 2
🍎 Counted! ID: 958 | Total: 3
🍎 Counted! ID: 1107 | Total: 4
⏳ Processed Frame 60 | Current Count: 4
🍎 Counted! ID: 1075 | Total: 5
🍎 Counted! ID: 1036 | Total: 6
🍎 Counted! ID: 907 | Total: 7
⏳ Processed Frame 90 | Current Count: 7
🍎 Counted! ID: 1175 | Total: 8
🍎 Counted! ID: 1098 | Total: 9
⏳ Processed Frame 120 | Current Count: 9
🍎 Counted! ID: 1150 | Total: 10
🍎 Counted! ID: 1106 | Total: 11
⏳ Processed Frame 150 | Current Count: 11
🍎 Counted! ID: 1105 | Total: 12
🍎 Counted! ID: 1124 | Total: 13
🍎 Counted! ID: 1109 | Total: 14
⏳ Processed Frame 180 | Current Count: 14
🍎 Counted! ID: 1090 | Total: 15
🍎 Counted! ID: 1392 | Total: 16
⏳ Processed Frame 210 | Current Count: 16
🍎 Counted! ID: 1128 | Total: 17
🍎 Counted! ID: 1156 | Total: 18
🍎 Counted! ID: 1445 | Total: 19
⏳ Proces

## STEP 6: Download & Result Verification

Since Colab cannot play videos directly with cv2.imshow, we provide a helper to download the result.

If you use local Notebook (i.e. Anaconda Jupyter Notebook), you can use another show method

In [9]:
# STEP 6: Download the Result Video
from google.colab import files

print(f"📂 Preparing download for: {output_video_path}")

try:
    files.download(output_video_path)
    print("✅ Download started automatically.")
except Exception as e:
    print(f"⚠️ Auto-download failed: {e}")
    print("👉 Please download manually from the 'Files' sidebar on the left.")

📂 Preparing download for: /content/final_smart_counter.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download started automatically.
